In [18]:
import sounddevice as sd
import soundfile as sf

In [19]:
channels = 1
data_type = 'float32'
data, fs = sf.read('f2.wav', dtype='float32')
blocksize = int(fs/10)
total_time = len(data)/fs

In [25]:
import argparse
import queue
import sys

import sounddevice as sd
import soundfile as sf


buffersize = 10

q = queue.Queue(maxsize=buffersize)


def callback(outdata, frames, time, status):
    if status.output_underflow:
        print('Output underflow: increase blocksize?', file=sys.stderr)
        raise sd.CallbackAbort
    assert not status
    try:
        data = q.get_nowait()
    except queue.Empty:
        print('Buffer is empty: increase buffersize?', file=sys.stderr)
        raise sd.CallbackAbort
    if len(data) < len(outdata):
        outdata[:len(data)] = data
        outdata[len(data):] = b'\x00' * (len(outdata) - len(data))
        raise sd.CallbackStop
    else:
        outdata[:] = data


try:
    with sf.SoundFile('f2.wav') as f:
        for _ in range(buffersize):
            data = f.buffer_read(blocksize, dtype=data_type)
            if not data:
                break
            q.put_nowait(data)  # Pre-fill queue
        stream = sd.RawOutputStream(
            samplerate=fs, blocksize=blocksize, type=data_type,
            callback=callback, finished_callback=event.set)
        with stream:
            timeout = blocksize * buffersize / fs
            while data:
                data = f.buffer_read(blocksize, dtype=data_type)
                q.put(data, timeout=timeout)
except KeyboardInterrupt:
    pass
except queue.Full:
    # A timeout occurred, i.e. there was an error in the callback
    parser.exit(1)
except Exception as e:
    pass

In [16]:
sd.play(data, fs)

In [ ]:
def callback(indata, outdata, frames, time, status):
    if status:
        print(status)

s = sd.Stream(channels=channels,
              callback=callback,
              dtype=data_type,
              blocksize=block_size,
              samplerate=fs)

with s:
    sd.sleep(int(total_time * 1000))

In [11]:
data.shape[0]/44100

11.97278911564626